# Week 9 Study Notebook

# UC San Diego OMDS DSC 207

## Assignment Instructions-
1. You may use any visualization tool - seaborn/matplotlib for a task
2. Sample outputs are provided but your output may vary based on choice of library and parameters.
3. Use the mandatory parameter values wherever specified

Note: Currently, there is an autograder configured for the week 9 assignment on Gradescope. Please feel free to ignore this as we will be grading the entire assignment manually.

# Natural Language Processing with `nltk`

`nltk` is the most popular Python package for Natural Language processing, it provides algorithms for importing, cleaning, pre-processing text data in human language and then apply computational linguistics algorithms like sentiment analysis.

It also includes many easy-to-use datasets in the `nltk.corpus` package, we can download for example the `movie_reviews` package using the `nltk.download` function:

## Inspect the Movie Reviews Dataset

In [ ]:
# Uncomment the below line and run this cell if you need to install nltk
#!pip install nltk

In [20]:
#Run this cell for all the imports
import pandas as pd
import numpy as np
import nltk

In [21]:
# Import the sentiment intensity analyzer
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [22]:
#Run this cell to download the dataset
nltk.download("movie_reviews")

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

You can also list and download other datasets interactively, just type:

`nltk.download()`
    
in the Jupyter Notebook.

In [23]:
#Run this cell to import the dataset
from nltk.corpus import movie_reviews

In [24]:
#Run this cell for later use in tokenization
nltk.download('vader_lexicon')  # for sentiment analysis
nltk.download('punkt_tab')  # for tokenizing

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## 1. Tokenize Text in Words

In [25]:
#Run this cell
romeo_text = """Why then, O brawling love! O loving hate!
O any thing, of nothing first create!
O heavy lightness, serious vanity,
Misshapen chaos of well-seeming forms,
Feather of lead, bright smoke, cold fire, sick health,
Still-waking sleep, that is not what it is!
This love feel I, that feel no love in this."""

The first step in Natural Language processing is generally to split the text into words, this process might appear simple but it is very tedious to handle all corner cases, see for example all the issues with punctuation we have to solve if we just start with a split on whitespace.

1.1 **Split `romeo_text` by spaces and store the resultant list of words in the variable `romeo_tokens`** [0.5 pt]

In [26]:
romeo_tokens = romeo_text.split()

In [27]:
assert type(romeo_tokens) == list
assert len(romeo_tokens) == 52

`nltk` has a sophisticated word tokenizer trained on English named `punkt` which we imported earlier in the notebook.

1.2  **Use the `nltk.word_tokenize(text)` function to properly tokenize `romeo_text` and stores the result as `romeo_words`. Print the resultant list.** Compare it to the whitespace splitting we used above and mention the difference. [0.5 pt]

In [28]:
romeo_words = nltk.word_tokenize(romeo_text)

In [29]:
assert type(romeo_words) == list
assert len(romeo_words) == 68

## 2. Build a bag-of-words model

The simplest model for analyzing text is just to think about text as an unordered collection of words (bag-of-words). This can generally allow to infer from the text the category, the topic or the sentiment.

From the bag-of-words model we can build features to be used by a classifier, here we assume that each word is a feature that can either be `True` or `False`.
We implement this in Python as a dictionary where for each word in a sentence we associate `True`.

2.1 **Write a function `build_bag_of_words(words)` that returns such a dictionary with {word : True} format given a set of words. Call the function with `romeo_words` and store the resultant dictionary as `romeo_word_dict`.** [1 pt]

In [32]:

def build_bag_of_words_features(words):
    word_dict = {word:True for word in words}
    return word_dict


romeo_word_dict = build_bag_of_words_features(romeo_words)

In [33]:
# Sanity check
assert type(build_bag_of_words_features(romeo_words)) == dict
assert sum(value for value in romeo_word_dict.values() if value) == 45

This is what we wanted, but we notice that also punctuation like "!" and words useless for classification purposes like "of" or "that" are also included.
Those words are named "stopwords" and `nltk` has a convenient corpus we can download:

In [34]:
#Run this cell
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
#Run this cell
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

Using the Python `string.punctuation` list and the English stopwords we can build better features by filtering out those words that would not help in the classification.

2.2 **Create a list `useless_words` that is a collection of stopwords in english and the punctuation characters.** [0.5 pt]

In [36]:
useless_words = nltk.corpus.stopwords.words("english") + list(string.punctuation)

In [37]:
assert type(useless_words) == list
assert len(useless_words) == 211

2.3 **Write a function `build_bag_of_words_features_filtered(words)` that returns a filtered bag of words - a dictionary with only useful words as key and 1 as the value. Call this function with `romeo_words` and store the resultant dictionary as `romeo_useful_word_dict`.** [1 pt]

In [40]:
def build_bag_of_words_features_filtered(words):
    return {word:1 for word in words if not word in useless_words}
romeo_useful_word_dict = build_bag_of_words_features_filtered(romeo_words)

In [41]:
# Sanity check
assert type(build_bag_of_words_features_filtered(romeo_words)) == dict
assert len(romeo_useful_word_dict) == 31

## 3. Frequencies of Words

It is common to explore a dataset before starting the analysis, in this section we will find the most common words and plot their frequency.

3.1. Using the `movie_reviews.words()` (the nltk corpus we imported previously) with no argument we can extract the words from the entire dataset as `all_words` and check that it is about 1.6 millions. [0.5 pt]

In [45]:
all_words = movie_reviews.words()
len(all_words)

1583820

3.2. Filter out `useless_words` as defined in the previous section, and create a new list `filtered_words` this will reduce the length of the dataset by more than a factor of 2. (Hint - Use python list comprehension) [0.5 pt]

In [58]:
filtered_words = [word for word in all_words if word not in useless_words]

In [59]:
len(filtered_words)

710579

In [60]:
assert type(filtered_words) == list

The `collection` package of the standard library contains a `Counter` class that is handy for counting frequencies of words in our list:

In [61]:
# Run this cell
from collections import Counter
word_counter = Counter(filtered_words)

3.3 It also has a [most_common() ](https://pythontic.com/containers/counter/most_common) method of the word_counter and store the top 10 used words from the corpus in `most_common_words`. [0.5 pt]?

In [64]:
most_common_words = word_counter.most_common(10)

In [65]:
print(most_common_words)

[('film', 9517), ('one', 5852), ('movie', 5771), ('like', 3690), ('even', 2565), ('good', 2411), ('time', 2411), ('story', 2169), ('would', 2109), ('much', 2049)]


In [66]:
assert type(most_common_words) == list
assert len(most_common_words) == 10

<!-- BEGIN QUESTION -->

## 4. Sentiment Analysis [2 pt]

Using the sentiment intensity analyzer, loop over the `list_sentences` and print the polarity scores of each of the sentence. (Hint - refer to lecture notebook)

In [69]:

list_sentences = ["Hello, how are you?", "Today is a nice day", "I don't like the food at the cafe", "This is the worst pizza I have ever had.", "The orange juice is delicious!", "I am late to class." ]
...

In [70]:
analyzer = SentimentIntensityAnalyzer()

for text in list_sentences:
  scores = analyzer.polarity_scores(text)
  compound_score = scores['compound']

  print(f'Text: {text}.\n, Scores: {scores}')

Text: Hello, how are you?.
, Scores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Text: Today is a nice day.
, Scores: {'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.4215}
Text: I don't like the food at the cafe.
, Scores: {'neg': 0.232, 'neu': 0.768, 'pos': 0.0, 'compound': -0.2755}
Text: This is the worst pizza I have ever had..
, Scores: {'neg': 0.339, 'neu': 0.661, 'pos': 0.0, 'compound': -0.6249}
Text: The orange juice is delicious!.
, Scores: {'neg': 0.0, 'neu': 0.501, 'pos': 0.499, 'compound': 0.6114}
Text: I am late to class..
, Scores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
